In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import numpy as np

# Prevents crashes when showing graphs
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
SIM_STEPS = 201

torch.manual_seed(42)
np.random.seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [2]:
# Pre-Determined train/test/val split
train_sims = np.load("../train_sims.npy")
train_sims = train_sims[train_sims < 750]
val_sims = np.load("../val_sims.npy")
val_sims = val_sims[val_sims < 750]
test_sims = np.load("../test_sims.npy")
test_sims = test_sims[test_sims < 750]

# Get porosity phi
def get_phi(sim,step):
    return cv2.imread(f"../Data200x200_withinfo/Image-{sim}-{step}_phi.jpg", cv2.IMREAD_GRAYSCALE)

# Get pressure
def get_pres(sim,step):
    return cv2.imread(f"../Data200x200_withinfo/Image-{sim}-{step}_P.jpg", cv2.IMREAD_GRAYSCALE)

# Get conductivity K
def get_k(sim,step):
    return cv2.imread(f"../Data200x200_withinfo/Image-{sim}-{step}_K.jpg", cv2.IMREAD_GRAYSCALE)

# Get all 3 as a 3-channel matrix
def get_all(sim,step):
    return np.array((get_k(sim,step), get_pres(sim,step), get_phi(sim,step)), dtype=np.float32)

In [3]:
# Darcy loss function
def darcy_loss(model, inp):
    # Takes in the k,pres,phi and outputs the prediction across the image.
    inp = inp.requires_grad_(True)
    out = model(inp)
    # out is in order K,P,phi, (conductivity, pressure, porosity)

    # Impose high pressure along the entire upper line by setting the pressure channelt to 200.
    out[:, 1:2, 0, :] = 200

    # If we assume the output is in order k,pres,phi
    # pres_grad is the gradient of the pressure along the y and x directions as a tuple
    pres_grad = torch.gradient(out[:, 1:2], dim=(-2,-1))

    # get velocity by multiplying the gradient by the conductivity
    y_grad = pres_grad[0] * out[:, 0:1]
    x_grad = pres_grad[1] * out[:, 0:1]

    # compute the divergence by the second derivative of the gradients and adding them together
    yy_grad = torch.gradient(y_grad, spacing=(1,),dim=(-2,))[0]
    xx_grad = torch.gradient(x_grad, spacing=(1,),dim=(-1,))[0]
    final = yy_grad + xx_grad

    # total divergence should be 0
    loss = (final**2).mean()

    return loss, out

In [4]:
# Building Blocks of the Unet

class TwoConv(nn.Module):
    # Basic block with 2 convolutional layers, each with a batch norm and relu
    
    def __init__(self, in_channels, out_channels, no_end_relu=False):
        super().__init__()
        
        self.seq = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

        if no_end_relu:
            self.seq = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1),
            )

    def forward(self, inp):
        return self.seq(inp)

# A single conv layer that will increase the height and width of the matrix by 2 each.
class SmallUp(nn.Module):

    def __init__(self, channels):
        super().__init__()

        self.conv = nn.Conv2d(channels, channels, 3, 1, 2)

    def forward(self, inp):
        return F.relu(self.conv(inp))

# A single conv layer that will decrease the height and width of the matrix by 2 each.
class SmallDown(nn.Module):

    def __init__(self, channels):
        super().__init__()

        self.conv = nn.Conv2d(channels, channels, 3, 1, 0)

    def forward(self, inp):
        return F.relu(self.conv(inp))
    
# Applies two convolutional layers, then pools
class Downsample(nn.Module):
    
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.conv = TwoConv(in_channels, out_channels)
        self.pool = nn.MaxPool2d(2)

    def forward(self, inp):

        return self.pool(self.conv(inp))

# Upsamples and concatenates the upsampled matrix with the "across" then performs convolution on the result
class Upsample(nn.Module):

    def __init__(self, in_channels, out_channels, tweak=None):
        super().__init__()
        # Upsamples by 2x
        self.up = nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1)
        self.tweak = tweak
        self.conv_after = TwoConv(in_channels, out_channels)

    def forward(self, below, across):
        # First upsample by 2x
        upsampled = self.up(below)
        # If tweak is active, apply it first
        if not self.tweak == None:
            upsampled = self.tweak(upsampled)
        # Concatenate with the same size on the downswing of the unet
        concat = torch.concat((upsampled, across), dim=-3)
        # Convolute them together
        return self.conv_after(concat)
    
# Define the actual model used
class SmallUnet(nn.Module):
    
    def __init__(self):
        super().__init__()

        # Input is Nx3x200x200
        self.c1 = TwoConv(3, 8)
        self.d1 = Downsample(8,16) # 16x100x100
        self.d2 = Downsample(16,32) # 32x50x50
        self.su = nn.Sequential(
            SmallUp(32),
            SmallUp(32),
            SmallUp(32)
        ) # 3x56x56
        self.d3 = Downsample(32,64) # 64x28x28
        self.d4 = Downsample(64,128) # 128x14x14
        self.d5 = Downsample(128, 256) # 256x7x7

        # Now back up
        self.u1 = Upsample(256, 128) # 128x14x14
        self.u2 = Upsample(128, 64) # 64x28x28
        self.u3 = Upsample(64, 32, tweak=nn.Sequential(
            SmallDown(32),
            SmallDown(32),
            SmallDown(32)
        ))  # 32x50x50
        self.u4 = Upsample(32,16) # 16x100x100
        self.u5 = Upsample(16,8) # 8x200x200
        self.final = TwoConv(8, 3, no_end_relu=True)

    def forward(self, input):
        # Start with convolution, expand 3 channels to 8.
        # Then downsample 5 times, saving the result
        top = self.c1(input)
        x1 = self.d1(top)
        x2 = self.d2(x1)
        x3 = self.d3(self.su(x2)) # Here we upsample slightly so that we can downsample with less border artifacts
        x4 = self.d4(x3)
        x5 = self.d5(x4)
        # Now that we're at 256x7x7, we upsample from here.
        # At each layer with concatenate with the xi that is the same size as the up after upsampling.
        up = self.u1(x5, x4)
        up = self.u2(up, x3)
        up = self.u3(up, x2) # Again, a small downsample here to get back on the proper resolution
        up = self.u4(up, x1)
        up = self.u5(up, top)
        # One last convolution on the result to return to 3 channels from 8, leaving us with the proper 3x200x200
        return self.final(up)

In [5]:
class MaskedDataset(torch.utils.data.Dataset):

    def __init__(self,
                 sims,
                 unmask_size=20,
                 points = None,
                 block_size = 50,
                 reveal_strategy = "block",
                 n_points = 200,
                 radius = 2,
                 steps = None,
                 H=200,
                 W=200,
                 channels="all",
                 mixed=False,
                 types=None,
                 noise=5,
                 return_mask=False,                 # allows visualiztion of mask
                 reveal_dim=[[(0, 1)], [(0, 1)]],   # x,y range for disks to exist
                 jitter_std=0.0,                    # % each disk drifts from deterministic position
                 deterministic_mask=True,            # if True, mask is deterministic and noise is 0
                 future_delta=0
                 ):
        
        self.sims = sims
        self.points = points
        self.steps = steps
        self.size = unmask_size
        self.reveal_strategy = reveal_strategy
        self.block_size = block_size
        self.n_points = n_points
        self.radius = radius
        self.H, self.W = H, W
        self.channels = channels
        self.mixed = mixed
        self.types = types
        self.noise = noise
        self.return_mask = return_mask
        self.reveal_dim = reveal_dim
        self.jitter_std = jitter_std
        self.deterministic_mask = deterministic_mask
        self.future_delta = future_delta

    def _chan_idx(self):
        if self.channels == "all":
            return [0,1,2]
        elif self.channels == "K":
            return [0]
        elif self.channels == "P":
            return [1]
        elif self.channels == "phi":
            return [2]
        else:
            raise ValueError("channels must be 'all', 'K', 'P', or 'phi'")

    
    

    def __getitem__(self, index):
        
        # --- In Progress --- 

            # pick a valid step
        if not isinstance(self.steps, np.ndarray):
            max_start = SIM_STEPS - 1 - self.future_delta  # ensures step + delta ≤ 199
            step = np.random.randint(1, max_start + 1)  
        else:
            step = int(self.steps[index])
            max_start = SIM_STEPS - 1 - self.future_delta
            if step > max_start:
                step = max_start



        # if not type(self.steps) == np.ndarray:
        #     step = np.random.randint(1,200)
        # else:
        #     step = self.steps[index]



        # Create tensor for the target
        t_cur = torch.tensor(get_all(self.sims[index], step), dtype=torch.float32)

        # Create 0 matrix
        z = torch.zeros_like(t_cur)

        # build a boolean mask of revealed pixels, shape (H,W)
        mask = torch.zeros((self.H, self.W), dtype=torch.bool)

        chans = self._chan_idx()

        if self.reveal_strategy == "block":
            # choose top-left for the block
            if not type(self.points) == np.ndarray:
                i0 = np.random.randint(0, self.H - self.block_size + 1)
                j0 = np.random.randint(0, self.W - self.block_size + 1)
            else:
                i0, j0 = self.points[index]
                i0 = max(0, min(i0, self.H - self.block_size))
                j0 = max(0, min(j0, self.W - self.block_size))
            mask[i0:i0+self.block_size, j0:j0+self.block_size] = True

        elif self.reveal_strategy == "disks":

            # used for reveal_dim
            # map fraction [0,1] to pixel indices [0, N-1] in mask layer
            def _segments_to_indices(segments, N, pad=0):
                idxs = []
                for a, b in segments:
                    i0 = max(pad, int(round(a * (N - 1))))
                    i1 = min((N - 1) - pad, int(round(b * (N - 1))))
                    if i1 >= i0:
                        idxs.append(torch.arange(i0, i1 + 1, dtype=torch.long))
                if not idxs:
                    # fallback to full range
                    return torch.arange(pad, N - pad, dtype=torch.long)
                return torch.unique(torch.cat(idxs)).to(torch.long)

            row_fracs = self.reveal_dim[0] # e.g, [(0, 1)]
            col_fracs = self.reveal_dim[1] # e.g, [(0, 1)]
            row_allowed = _segments_to_indices(row_fracs, self.H, pad=self.radius)
            col_allowed = _segments_to_indices(col_fracs, self.W, pad=self.radius)

            # choose grid shape close to aspect ratio 
            # works with non-squares
            Hspan = (row_allowed[-1] - row_allowed[0] + 1) if len(row_allowed) > 0 else self.H
            Wspan = (col_allowed[-1] - col_allowed[0] + 1) if len(col_allowed) > 0 else self.W
            ratio = float(Wspan) / max(1.0, float(Hspan))
            ny = int(max(1, round(np.sqrt(self.n_points / max(1e-8, ratio)))))
            nx = int(max(1, round(self.n_points / ny)))
            while nx * ny < self.n_points:
                nx += 1

            # pick evenly spaced indices from rows/cols allowed
            def pick_lin_indices(allowed, k):
                if k <= 1:
                    return allowed[len(allowed)//2]
                pos = torch.linspace(0, len(allowed)-1, steps=k)
                idx = torch.round(pos).long()
                return allowed[idx]
            
            
            row_picks = pick_lin_indices(row_allowed, ny)
            col_picks = pick_lin_indices(col_allowed, nx)
            yy, xx = torch.meshgrid(row_picks, col_picks, indexing="ij")
            points = torch.stack([yy.reshape(-1), xx.reshape(-1)], dim=1) # (ny*nx, 2)
            
            # if more than n_points, subselect
            if points.shape[0] > self.n_points:
                sel_pos = torch.linspace(0, points.shape[0]-1, steps=self.n_points)
                sel_idx = torch.round(sel_pos).long()
                points = points[sel_idx]

            ii = points[:, 0]
            jj = points[:, 1]

            if not self.deterministic_mask:
                if self.jitter_std is not None and self.jitter_std > 0:
                    # convert std (like 0.01 of image size) to pixels
                    sigmaH = float(self.jitter_std) * self.H
                    sigmaW = float(self.jitter_std) * self.W
                    
                    # Add Gaussian noise in pixel units
                    ii = ii.to(torch.float32) + torch.randn_like(ii, dtype=torch.float32) * sigmaH
                    jj = jj.to(torch.float32) + torch.randn_like(jj, dtype=torch.float32) * sigmaW

                    # Round and clamp so they stay inside bounds
                    ii = ii.round().clamp(self.radius, self.H - 1 - self.radius).to(torch.long)
                    jj = jj.round().clamp(self.radius, self.W - 1 - self.radius).to(torch.long)




            yy, xx = torch.meshgrid(torch.arange(self.H), torch.arange(self.W), indexing="ij")
            for y0, x0 in zip(ii, jj):
                disk = (yy - int(y0))**2 + (xx - int(x0))**2 <= (self.radius**2)
                mask |= disk
        else:
            raise ValueError(f"Unknown reveal_strategy: {self.reveal_strategy}")
        

        obs = t_cur[chans].clone()
        # Add noise (0 - 255 scale)
        if self.noise is not None and self.noise > 0:
            sigma = float(self.noise)
            obs = obs + sigma * torch.randn_like(obs)
            obs.clamp_(0.0, 255.0)


        z[chans, :, :] = torch.where(mask, obs, torch.zeros_like(obs))








        # --- In Progress --- 

        if self.future_delta > 0:
            step_f = step + self.future_delta   
            t_label = torch.tensor(get_all(self.sims[index], step_f), dtype=torch.float32)
        else:
            t_label = t_cur.clone()



        if self.return_mask:
            return z,t_label, mask
        else:  
            return z,t_label
    
    def __len__(self):
        return self.sims.shape[0]


In [6]:
import numpy as np
import torch

def build_val_dataset_from_meta(meta):
    """
    Rebuilds the validation MaskedDataset exactly as used in training.
    Assumes you saved meta with keys like:
      - reveal_strategy, n_points, radius, noise, channels
      - val_steps, val_points
      - reveal_dim, deterministic_mask_val, jitter_std_val
      - mixed, future_delta
    """
    reveal_strategy   = str(meta["reveal_strategy"])
    n_points          = int(meta["n_points"])
    radius            = int(meta["radius"])
    noise             = float(meta["noise"])
    channels          = meta["channels"]
    val_steps         = meta["val_steps"]
    val_points        = meta["val_points"]
    reveal_dim        = meta["reveal_dim"].tolist() if hasattr(meta["reveal_dim"], "tolist") else meta["reveal_dim"]
    deterministic_mask= bool(meta["deterministic_mask_val"])
    jitter_std        = float(meta["jitter_std_val"])
    mixed             = bool(meta["mixed"])
    future_delta      = int(meta["future_delta"]) if "future_delta" in meta.files else 0

    # match plotting/val reproducibility
    torch.manual_seed(123); np.random.seed(123)

    val_data = MaskedDataset(
        val_sims,
        reveal_strategy=reveal_strategy,
        n_points=n_points,
        radius=radius,
        noise=noise,
        channels=channels,
        points=val_points,
        steps=val_steps,
        reveal_dim=reveal_dim,
        deterministic_mask=deterministic_mask,
        jitter_std=jitter_std,
        mixed=mixed,
        future_delta=future_delta,
    )
    return val_data, val_steps, future_delta

In [7]:
def make_sample_for_sim_time(sim_id, t, meta, future_delta):
    """
    Build a single (z, t_label) pair for a specific sim and time t
    using the same config that was used during training.
    """
    reveal_strategy    = str(meta["reveal_strategy"])
    n_points           = int(meta["n_points"])
    radius             = int(meta["radius"])
    noise              = float(meta["noise"])
    channels           = meta["channels"]
    reveal_dim         = meta["reveal_dim"].tolist() if hasattr(meta["reveal_dim"], "tolist") else meta["reveal_dim"]
    deterministic_mask = bool(meta["deterministic_mask_val"])
    jitter_std         = float(meta["jitter_std_val"])
    mixed              = bool(meta["mixed"])

    # make sure t is valid for this delta
    SIM_STEPS = 201
    max_start = SIM_STEPS - 1 - future_delta   # same logic as in MaskedDataset
    t = min(int(t), max_start)

    ds = MaskedDataset(
        sims=np.array([sim_id]),
        reveal_strategy=reveal_strategy,
        n_points=n_points,
        radius=radius,
        noise=noise,
        channels=channels,
        points=None,                         # let it pick mask positions
        steps=np.array([t]),                 # <- force time t
        reveal_dim=reveal_dim,
        deterministic_mask=deterministic_mask,
        jitter_std=jitter_std,
        mixed=mixed,
        future_delta=future_delta,
    )

    z, t_label = ds[0]   # only one sample
    return z, t_label, t

In [8]:
vals = range(140)
f_delta_values = list(vals[::2])
print(f_delta_values)

tests = [
    {
        "name": f"full_d{d}_n16_clean",
        "reveal_strategy": "disks",
        "n_points": 16,
        "radius": 5,
        "mixed": False,
        "noise": 0, 
        "deterministic_mask": False,
        "jitter_std": 0.02,
        "reveal_dim": [[(0.2, .8)],[(.2,.8)]],
        "future_delta": d,
    }
    for d in f_delta_values
]

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 102, 104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128, 130, 132, 134, 136, 138]


In [9]:
import numpy as np
import torch

def side_from_image(img, channel=0, top_rows=40, min_mass=1e-3):
    """
    Decide whether the channel is more on the left or right in the top part
    of the image.

    img: (C, H, W) numpy array or torch tensor
    channel: which channel to look at (0=K, 1=P, 2=phi)
    top_rows: only look at rows [0:top_rows] to focus on the inlet region
    min_mass: if total intensity is below this, treat as 'too weak'
    """
    if torch.is_tensor(img):
        arr = img.detach().cpu().numpy()
    else:
        arr = np.asarray(img)

    C, H, W = arr.shape
    top_rows = min(top_rows, H)

    patch = arr[channel, :top_rows, :]          # (top_rows, W)

    left_mass  = patch[:, :W//2].sum()
    right_mass = patch[:, W//2:].sum()
    total      = left_mass + right_mass

    if total < min_mass:
        # essentially no channel signal -> skip
        return None, 0.0, total

    margin = abs(left_mass - right_mass) / max(total, 1e-8)
    side   = "L" if left_mass > right_mass else "R"

    return side, margin, total

In [10]:
def left_right_accuracy_over_time(
    model_name,
    base_path="",
    future_delta=None,        # if None, read from meta
    time_values=None,         # list of times t to evaluate
    channel=0,                # 0=K, 1=P, 2=phi (for deciding side)
    top_rows=40,
    min_mass=1e-3,
    strong_margin=0.10,       # margin threshold for “strong GT”
    max_sims=None,            # limit sims for speed
):
    """
    For a single trained model, compute directional accuracy (left vs right)
    as a function of the start time t.

    Returns a dict with:
      times, acc_all, acc_strong, n_all, n_strong,
      n_lowmass, n_ambig, future_delta
    """
    # ---- load meta + model ----
    meta_path  = f"{base_path}meta_{model_name}.npz"
    model_path = f"{base_path}{model_name}.pt"

    meta  = np.load(meta_path, allow_pickle=True)
    model = torch.load(model_path, weights_only=False,
                       map_location=torch.device("cpu")).to(device)
    model.eval()

    # get Δ
    if future_delta is None:
        future_delta = int(meta["future_delta"]) if "future_delta" in meta.files else 0

    # which sims to use
    sims = val_sims.copy()
    if max_sims is not None:
        sims = sims[:max_sims]

    # default time grid if not given
    if time_values is None:
        # e.g. from 60 to 140 in steps of 5
        time_values = list(range(60, 141, 5))

    results = {
        "times": [],
        "acc_all": [],
        "acc_strong": [],
        "n_all": [],
        "n_strong": [],
        "n_lowmass": [],
        "n_ambig": [],
        "future_delta": future_delta,
    }

    for t in time_values:
        # skip times whose future t+Δ goes past the end
        if t + future_delta >= SIM_STEPS:
            continue

        correct_all = total_all = 0
        correct_str = total_str = 0
        n_lowmass = 0
        n_ambig   = 0

        for sim_id in sims:
            sim_id = int(sim_id)

            # build masked input at time t using your same meta config
            z, t_label, t_actual = make_sample_for_sim_time(sim_id, t, meta, future_delta)

            t_f = t + future_delta

            # ground-truth future (from raw data)
            img_true_future = get_all(sim_id, t_f)    # (3, H, W)

            # model prediction
            with torch.no_grad():
                out = model(z.to(device).unsqueeze(0))[0]
            img_pred_future = out.detach().cpu().numpy()  # (3, H, W)

            # ---- decide L/R for GT ----
            side_true, margin_true, mass_true = side_from_image(
                img_true_future,
                channel=channel,
                top_rows=top_rows,
                min_mass=min_mass,
            )

            if side_true is None:
                # too little signal -> low mass
                n_lowmass += 1
                continue

            # ---- decide L/R for prediction ----
            side_pred, margin_pred, mass_pred = side_from_image(
                img_pred_future,
                channel=channel,
                top_rows=top_rows,
                min_mass=min_mass,
            )

            if side_pred is None:
                # model predicted nothing meaningful: count as trial, but incorrect
                total_all += 1
                # not strong GT if margin below threshold
                if margin_true >= strong_margin:
                    total_str += 1
                continue

            # classify GT as strong vs ambiguous
            is_strong = (margin_true >= strong_margin)
            is_ambig  = (margin_true < strong_margin)

            total_all += 1
            if side_pred == side_true:
                correct_all += 1

            if is_strong:
                total_str += 1
                if side_pred == side_true:
                    correct_str += 1
            else:
                n_ambig += 1

        acc_all = np.nan if total_all == 0 else correct_all / total_all
        acc_str = np.nan if total_str == 0 else correct_str / total_str

        results["times"].append(t)
        results["acc_all"].append(acc_all)
        results["acc_strong"].append(acc_str)
        results["n_all"].append(total_all)
        results["n_strong"].append(total_str)
        results["n_lowmass"].append(n_lowmass)
        results["n_ambig"].append(n_ambig)

        print(
            f"[{model_name}] t={t:3d}, Δ={future_delta:3d} | "
            f"acc_all={acc_all:.3f} (n={total_all}), "
            f"acc_strong={acc_str:.3f} (n_str={total_str}), "
            f"lowmass={n_lowmass}, ambig={n_ambig}"
        )

    return results

In [11]:
import matplotlib.pyplot as plt

def plot_lr_accuracy_curves(model_names, base_path="", time_values=None,
                            channel=0, top_rows=40, strong_margin=0.10,
                            max_sims=250):
    """
    model_names: list of strings, e.g. ["full_d0_n16_clean", "full_d20_n16_clean", ...]
    """
    plt.figure(figsize=(8,5))

    all_results = {}

    for name in model_names:
        print(f"\n=== Evaluating left/right vs time for {name} ===")
        res = left_right_accuracy_over_time(
            name,
            base_path=base_path,
            future_delta=None,          # read from meta
            time_values=time_values,    # you can pass something like range(70,121,5)
            channel=channel,
            top_rows=top_rows,
            strong_margin=strong_margin,
            max_sims=max_sims,
        )
        all_results[name] = res

        ts   = np.array(res["times"])
        accs = np.array(res["acc_strong"])   # I’d plot strong GT for clarity

        Δ = res["future_delta"]
        plt.plot(ts, accs, "-o", label=f"Δ={Δ}")

    plt.xlabel("Start time t")
    plt.ylabel("Directional accuracy (strong GT)")
    plt.title("Left/right prediction vs time for different Δ")
    plt.ylim(0, 1.05)
    plt.grid(True, alpha=0.3)
    plt.legend(title="Future Δ")
    plt.tight_layout()
    plt.show()

    return all_results

In [12]:
import os

for cfg in tests:
    name = cfg["name"]
    Δ    = cfg["future_delta"]

    # Required files for evaluation
    model_path = f"{name}.pt"
    meta_path  = f"meta_{name}.npz"

    # Check if both files exist
    if not os.path.exists(model_path):
        print(f"🚫 Skipping {name} — model file not found ({model_path})")
        continue
    if not os.path.exists(meta_path):
        print(f"🚫 Skipping {name} — meta file not found ({meta_path})")
        continue

    # Assign time grid based on future_delta
    if Δ <= 10:
        time_grid = list(range(80, 121, 5))   # short-horizon
    elif Δ <= 40:
        time_grid = list(range(60, 121, 5))   # medium-horizon
    else:
        time_grid = list(range(30, 121, 5))   # long-horizon

    print(f"➡ Evaluating {name} (Δ={Δ}) over times {time_grid}...")

    res = left_right_accuracy_over_time(
        name,
        base_path="",
        future_delta=None,      # read from meta
        time_values=time_grid,  # customized per Δ
        channel=0,
        top_rows=40,
        strong_margin=0.10,
        max_sims=250,
    )

    print(f"Done. Accuracy summary: {res.get('accuracy_summary', 'See metrics output')}")
    print("-" * 50)

➡ Evaluating full_d0_n16_clean (Δ=0) over times [80, 85, 90, 95, 100, 105, 110, 115, 120]...


KeyboardInterrupt: 